### Load the Commit Features

In [1]:
project_root_path = '../..'
lib_path = project_root_path + '/python/lib/'
import json
import sys
sys.path.append(lib_path)
import commit_features

In [2]:
import json
in_path = project_root_path + '/python/cfg/ds_pipeline.json'
config_map = json.loads(open(in_path, 'r').read())
config_map

cfg = config_map['r17_01']
cfg2 = config_map['r17_02']

In [3]:
project_stats_dir = project_root_path + '/' + cfg['stats_dir']
file_name = cfg['stat_file_name']
all_commit_features = commit_features.load_all(project_root_path, project_stats_dir, file_name)

In [4]:
X = commit_features.make_matrix(all_commit_features, cfg['classifier']['features'])

### Load the Trained Model

In [5]:
import pickle
model_path = project_root_path + '/' + cfg['classifier']['model_path']
fin = open(model_path, 'rb')
model_deserial = pickle.load(fin)
fin.close()
y_pred = model_deserial.predict(X)
print(len(y_pred))

477520


In [7]:
model2_path = project_root_path + '/' + cfg2['classifier']['model_path']
fin2 = open(model2_path, 'rb')
model2 = pickle.load(fin2)
fin2.close()
y2_pred = model2.predict(X)
print(len(y2_pred))

477520


### Compile Hacker Histories

In [8]:
hacker_histories = {}
for i in range(len(y_pred)):
    features = all_commit_features[i]
    pred = y_pred[i]
    pred2 = y2_pred[i]
    author = features['author']
    features['predicted_action'] = pred
    features['predicted_scope'] = pred2
    if author not in hacker_histories: hacker_histories[author] = []
    hacker_commits = hacker_histories[author]
    hacker_commits.append(features)

### Show Hacker Commit Types

In [11]:
import matplotlib.pyplot as plt

hacker_histograms = []
hacker_commit_counts = {}
for hacker in hacker_histories.keys():
    commits = hacker_histories[hacker]
    commit_actions = [commit['predicted_action'] for commit in commits]
    commit_scopes = [commit['predicted_scope'] for commit in commits]
    observed_actions = list(set(commit_actions))
    observed_scopes = list(set(commit_scopes))
    hacker_commit_actions = []
    hacker_commit_scopes = []
    action_count = 0
    for observed_action in observed_actions:
        observations = [1 for commit_action in commit_actions if commit_action == observed_action]
        count = len(observations)
        hacker_commit_actions.append(dict(num_commits=count, commit_action=observed_action))
        action_count += count
    scope_count = 0
    for observed_scope in observed_scopes:
        observations = [1 for commit_scope in commit_scopes if commit_scope == observed_scope]
        count = len(observations)
        hacker_commit_scopes.append(dict(num_commits=count, commit_scope=observed_scope))
        scope_count += count
    hacker_commit_counts[hacker] = action_count
    sorted_list = sorted(hacker_commit_actions, key=lambda d: d['num_commits'])
    sorted_list.reverse()
    hacker_histograms.append(dict(hacker=hacker, commit_count=action_count, commit_stats=sorted_list))

sorted_histograms = sorted(hacker_histograms, key=lambda d: d['commit_count'])
sorted_histograms.reverse()
for histo in sorted_histograms:
    if histo['commit_stats'][0]['num_commits'] > 2000:
        print(histo['hacker'])
        for row in histo['commit_stats']:
            print(str(row['num_commits']) + "\t" + str(row['commit_action']))
        print()
        break

Pradyun Gedam <pradyunsg@gmail.com>
5994	Python
1174	Other
1173	Pydata
285	Text
275	Yarn
188	Markdown
2	Interface



In [14]:
import json

do_write = False
if do_write:
    out_path = project_root_path + '/sandbox/data/bob/hacker_history-2022-05-21.json'
    out_handle = open(out_path, "w");
    out_handle.write(json.dumps(sorted_histograms, indent=2));
    out_handle.flush()
    out_handle.close()